# Lab S2 (6./7./9./10./ Dec 2021): Initial Ideas, First Tests

In [1]:
import numpy as np
import cv2
import json
import PIL
from matplotlib import image
from matplotlib import pyplot as plt

#### Thoughts and notes
- preprocessing should do color correction to deal with the different camera colors


#### stiching the images together


In [2]:
def integrate_timestep(timestep,path,homography,mask):
    #create a mask:
    image_mask = np.asarray(PIL.Image.open(mask))
    mask = np.zeros(shape=(1024,1024,3))
    #PIL.Image.fromarray(np.uint8(image_mask)).show()
    for i in range(3): 
        mask[:,:,i] = image_mask.astype(bool).copy()

    #create overlap mask:
    ov_mask=mask.copy()

    #final_integrated_image:
    int_im=np.zeros(shape=(len(mask),len(mask[0]),3))



    # Opening JSON file
    data = homography
    names=list(data.keys())


    #filter for timestep:
    index= list([True if item[0]==str(timestep) else False for i,item in enumerate(names)])
    filtered_names = [i for (i, v) in zip(names, index) if v]


    #apply homographies, the mask overlap and integrate images
    for i,name in enumerate(filtered_names):
        #load homography
        homography= np.array(data[name])

        #load image
        im_file_name=path+name+".png"
        im = np.asarray(PIL.Image.open(im_file_name))

        #mask out:
        im = np.multiply(im, mask) 

        #apply homography
        #PIL.Image.fromarray(np.uint8(im)).show()
        warped_image =cv2.warpPerspective(im,homography,im.shape[:2])
        #PIL.Image.fromarray(np.uint8(warped_image)).show()

        #adjust final overlapp mask
        ov_mask = np.where(warped_image > 0, ov_mask, 0)

        #integrate_images:
        int_im=int_im+warped_image

    #only areas where all images contributed count:
    int_im=np.multiply(int_im, ov_mask) 

    #normalize
    int_im=int_im/len(filtered_names)
    
    return int_im

In [6]:

path='data/train/train-1-9/'
f = open(path+"homographies.json")
homography = json.load(f)
f.close()
mask='data/mask.png'
results= integrate_timestep(2,path,homography,'data/mask.png')
PIL.Image.fromarray(np.uint8(results)).show()